# __Questions proposed:__

1. How many tomatoes were sold during Jan 1st, 2016?
2. Display the total paid amount for each product, display the product name.
3. How many customers buy more than two different products on every visit (i.e. day)?
4. Assuming a customer does not return the same day to buy the same product twice.How would you identify if the table contains duplicates?
5. Display the 2nd most paid product every day.

In [15]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('sale.db')
c = conn.cursor()

In [16]:
#How many tomatoes were sold during Jan 1st, 2016?

c.execute('''
SELECT date, SUM(units_sold) AS 'Tomatoes sold'
FROM Sales
WHERE sales.date = '2016-01-01' AND sales.product_id = (
    SELECT products.id
    FROM products
    WHERE products.name = 'Tomato'
    )
''')
tomatoes_1stjan = c.fetchall()
print('How many tomatoes were sold during Jan 1st, 2016?\n' + str(tomatoes_1stjan[0][1]))

How many tomatoes were sold during Jan 1st, 2016?
7


In [5]:
#Display the total paid amount for each product, display the product name.

c.execute('''
SELECT products.name AS 'Name', SUM(sales.paid_amount) AS  'Total paid amount for each product'
FROM Sales, Products
WHERE sales.product_id = products.id
GROUP BY product_id
''')
totals = c.fetchall()
print('\nTotal paid amount for each product and the product name:')
print(totals)


Total paid amount for each product and the product name:
[('Tomato', 123), ('Cucumber', 85), ('Avocado', 360), ('Red_Pepper', 60), ('Orange', 60)]


In [11]:
#How many customers buy more than two different products on every visit (i.e. day)?

c.execute('''
SELECT DISTINCT COUNT(product_id) count, customer_id, date
    FROM Sales
    GROUP BY date, customer_id
HAVING COUNT > 2
''')
more_2 = c.fetchall()
print('\nThe amount of customers that buy more than two different products on every visit(day):')
print(more_2)

#the answer is none


The amount of customers that buy more than two different products on every visit(day):
[]


In [13]:
# Assuming a customer does not return the same day to buy the same product twice.
# How would you identify if the table contains duplicates?

c.execute('''
SELECT date, customer_id, product_id, COUNT(*)
FROM sales
GROUP BY date, customer_id, product_id
HAVING COUNT(*) > 1;
''')
dupl = c.fetchall()
print('\nDuplicates:')
print(dupl)


Duplicates:
[('2016-01-14', 2, 3, 2)]


In [14]:
# Display the 2nd most paid product every day.
c.execute('''
SELECT date, product_id, MAX(soma)
FROM (
    SELECT a.date, a.product_id, SUM(a.paid_amount) soma
    FROM sales AS a, (
        SELECT MAX(soma), date, product_id
        FROM (
            SELECT SUM(paid_amount) soma, date, product_id
            FROM 'Sales'
            GROUP BY date, product_id
            ORDER BY date, soma DESC
            )
        GROUP BY date
        ) AS top1
    WHERE a.date = top1.date
    AND a.product_id <> top1.product_id
    GROUP BY a.date, a.product_id
    )
GROUP BY date
''')
# we get the first most paid, and exclude it from a new table
# and then get the max again (therefore, the 2nd highest)

# bette to use FROM wheter then NOT IN for performance
# FROM creates a temporary table to compar all info to
# NOT IN creates a new table for each registry comparison, making it slower

# WHERE date = date must be there to allow for the comparison to be filtered by day,
# otherwise the max value could be counted as being top 1 for the month, not each day
# it makes the restriction per day

top2day = c.fetchall()
print('\nSecond most sold product in each day:')
print(top2day)


Second most sold product in each day:
[('2016-01-01', 2, 35), ('2016-01-07', 3, 45), ('2016-01-10', 4, 60), ('2016-01-14', 1, 60)]
